# Contingency Tables (still not sure what that means)

#### Connect to the Database

In [1]:
import psycopg2 as pg
import pandas as pd

# Database setup
host = "localhost"
database = "cdm"
user = "postgres"
password = %env PGPASSWORD
connection_string = "host={} dbname={} user={} password={}".format(host, database, user, password)

db = pg.connect(connection_string)

### Privacy Functions

In [102]:
import numpy as np
def perturb_gender_decade(table, epsilon=0.1):
    for i, row in table.iterrows():
        noise = np.random.laplace(scale=1/epsilon);
        cur_count = np.round(row['count'] + noise)
        if cur_count < 0:
            cur_count = 0
        table.set_value(i, 'count', cur_count)
    return table

def perturb_gender_drug(table, epsilon=0.1):
    for i, row in table.iterrows():
        noise = np.random.laplace(scale=1/epsilon);
        cur_count = np.round(row['total'] + noise)
        if cur_count < 0:
            cur_count = 0
        table.set_value(i, 'total', cur_count)
    return table
    

## GROUP BY queries

### Heart Failure by Gender and Birth Decade

In [2]:
query = """
    SELECT
        extract('year' FROM date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth))) AS birth_decade,
        CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
        COUNT(*)
    FROM person p, condition_occurrence con_oc, concept con
    WHERE p.person_id = con_oc.person_id 
        AND con_oc.condition_concept_id = con.concept_id
        AND (con.concept_name LIKE '%Heart Failure%' OR
            con.concept_name LIKE '%heart failure%')
        AND con.domain_id = 'Condition'
        AND con.concept_class_id='Clinical Finding'
    GROUP BY
        date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth)),
        p.gender_concept_id
    ORDER BY
        birth_decade,
        gender;"""
results = pd.read_sql(query, con=db)
#results_perturb = perturb_gender_decade(results)

df = results.pivot(index='gender', columns='birth_decade', values='count')

df.columns.names = ['Birth Decade']
df.index.names = ['Gender']

heart_failure = df
heart_failure

Birth Decade,1900.0,1910.0,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0
Gender,,,,,,,,,
Female,2988,33350,99322,113796,59256,18212,11294,4540,1392
Male,1156,14692,66048,93864,53006,18256,10334,4942,1152


### HIV by Gender and Birth Decade

In [99]:
query = """
SELECT
  extract('year' FROM date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth))) AS birth_decade,
  CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
  COUNT(*)
FROM
  condition_occurrence co,
  person p
WHERE
  co.person_id = p.person_id AND
  condition_concept_id = '4241530'
GROUP BY 
  date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth)),
  p.gender_concept_id
ORDER BY
  birth_decade,
  gender;"""
results = pd.read_sql(query, con=db)
#results_perturb = perturb_gender_decade(results)

df = results.pivot(index='gender', columns='birth_decade', values='count')
df.columns.names = ['Birth Decade']
df.index.names = ['Gender']

HIV = df
HIV

Birth Decade,1900.0,1910.0,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0
Gender,,,,,,,,,
Female,12.0,164.0,442.0,604.0,374.0,110.0,128.0,38.0,16.0
Male,NaN,70.0,262.0,476.0,266.0,164.0,92.0,48.0,14.0


### Diabetes by Gender and Age

In [4]:
query = """
    SELECT
        extract('year' FROM date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth))) AS birth_decade,
        CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
        COUNT(*)
    FROM person p, condition_occurrence con_oc, concept con
    WHERE p.person_id = con_oc.person_id 
        AND con_oc.condition_concept_id = con.concept_id
        AND con.concept_class_id='Clinical Finding'
        AND con_oc.condition_concept_id=con.concept_id
        AND (con.concept_name LIKE '%Diabetes%' OR
            con.concept_name LIKE '%diabetes%')
        AND con.domain_id = 'Condition'
    GROUP BY
        date_trunc('decade', make_date(p.year_of_birth, p.month_of_birth, p.day_of_birth)),
        p.gender_concept_id
    ORDER BY
        birth_decade,
        gender;"""
results = pd.read_sql(query, con=db)
#results_perturb = perturb_gender_decade(results)

df = results.pivot(index='gender', columns='birth_decade', values='count')

df.columns.names = ['Birth Decade']
df.index.names = ['Gender']

diabetes = df
diabetes

Birth Decade,1900.0,1910.0,1920.0,1930.0,1940.0,1950.0,1960.0,1970.0,1980.0
Gender,,,,,,,,,
Female,7450,83022,259308,336062,181384,50166,29618,14730,4270
Male,2316,32368,163328,260810,154894,48180,29416,13820,3994


### Drugs by Gender

In [101]:
query = """
SELECT
  c.concept_name,
  CASE p.gender_concept_id WHEN 8532 THEN 'Female' ELSE 'Male' END AS gender,
  count(*) AS total
FROM
  person p,
  drug_exposure de,
  top_drugs td,
  concept c
WHERE
  c.concept_id = de.drug_concept_id AND
  de.drug_concept_id = td.concept_id AND
  de.person_id = p.person_id
GROUP BY
  c.concept_name,
  gender
ORDER BY 
  c.concept_name,
  gender;
"""

results = pd.read_sql(query, con=db)

df = results.pivot(index='gender', columns='concept_name', values='total')

df.columns.names = ['Drug Name']
df.index.names = ['Gender']

drugs = df
drugs

Drug Name,Epoetin Alfa,Gemfibrozil 600 MG Oral Tablet,"Influenza virus vaccine, trivalent (IIV3), split virus, 0.5 mL dosage, for intramuscular use",Lovastatin 20 MG Oral Tablet,Omeprazole 20 MG Delayed Release Oral Capsule,Oxygen 99 % Gas for Inhalation,Simvastatin 40 MG Oral Tablet,paricalcitol Injectable Solution
Gender,,,,,,,,
Female,83344,22417,43881,19364,22380,37516,17637,45392
Male,55902,15008,33530,12676,13868,23943,11772,30355


In [108]:
query = """
SELECT 
  sc.count AS stay_length,
  count(*) AS num_patients
FROM
  stay_count sc
GROUP BY
  stay_length
ORDER BY
  stay_length;
"""

results = pd.read_sql(query, con=db)
results = results.set_index('stay_length')
results.columns = ['Stay Length', 'Number of Patients']

results

ValueError: Length mismatch: Expected axis has 1 elements, new values have 2 elements